- 실습중인 쇼핑몰 상품은 여러페이지에 걸쳐 진열되어 있음
    - 함수로 구성 후 여러페이지 크롤링

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
# context = ssl._create_unverified_context()
url="http://jolse.com/category/toners-mists/1019/"

### 함수를 기능별로 세분화
- 요청 및 파싱 객체 생성 : get_request_product(url), 반환값-bs4 객체
- 전체 정보 추출 : get_page_product(bs_obj)
    - bs_obj를 전달 받아서
    - get_product_info(box)를 호출(제품1개에 대한 정보가 반환)
    - df를 생성 후 저장
- 각 제품의 정보 추출 : get_product_info(box)
    - 제품정보 1개에 대해 제품명/가격/세일가격을 추출해서 반환

In [41]:
prd_dict = {'품목':[],
           '가격':[],
           '세일가격':[]}
df_fin = pd.DataFrame(prd_dict)

In [10]:
# 요청 및 파싱 객체 생성
def get_request_product(url) :
    # 1. 소스추출
    url = 'http://jolse.com/category/toners-mists/1019/'
    htmls = urlopen(url)
    htmls = htmls.read()
    # 2. 파싱객체 생성
    bs_obj = BeautifulSoup(htmls, 'html.parser')
    return bs_obj

In [29]:
def get_page_product(url) :
    global df_fin # 전역변수임을 알려줌
    bs_obj = get_request_product(url)
    # 페이지내 전체 제품정보 추출
    boxes = bs_obj.findAll('div',{'class':'description'})
    # 각 제품에 대한 정보 추출
    for box in boxes :
        res = pd.DataFrame(pd.DataFrame(get_product_info(box),index=range(1,2)))
        df_fin = pd.concat([df_fin,res],axis=0,ignore_index=True)

In [31]:
def get_product_info(box) :
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    # 가격
    price = box.find('ul').findAll('span')[1].text
    sale_price = box.find('ul').findAll('span')[-1].text
    return {'품목':prd_name,'가격':price,'세일가격':sale_price}

In [13]:
url

'http://jolse.com/category/toners-mists/1019/'

In [33]:
get_page_product(url)

In [34]:
df_fin

,품목,가격,세일가격
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,[+FREE GIFT♥] Haruharu WONDER Black Rice Hyal...,USD 37.00,USD 15.99
5,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 16.15
6,SOME BY MI AHA BHA PHA 30 Days Miracle Toner ...,USD 24.00,USD 17.40
7,Pyunkang Yul Essence Toner 100ml,USD 14.50,USD 9.89
8,secretKey Hyaluron Aqua Soft Toner 500ml,USD 22.00,USD 18.70
9,ROUND LAB 1025 Dokdo Toner 200ml,USD 17.00,USD 14.45


## 여러 페이지 추출
- 해당 쇼핑몰의 url 분석
- 각 페이지에 대하여 url에 파라미터가 다르게 전송됨
- https://jolse.com/category/toners-mists/1019?page= + 페이지번호
- url 작성시 페이지번호를 따로 부착

In [42]:
base_url = 'https://jolse.com/category/toners-mists/1019?page='
for i in range(1,5) :
    url = base_url + str(i)
    get_page_product(url)

In [43]:
df_fin

,품목,가격,세일가격
0,Abib Heartleaf Calming Toner Skin Booster 200ml,USD 21.00,USD 14.70
1,SOME BY MI Propolis B5 Glow Barrier Calming T...,USD 22.00,USD 17.60
2,Cos De BAHA Centella Facial Toner 200ml,USD 16.45,USD 13.98
3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,USD 37.00,USD 31.45
4,[+FREE GIFT♥] Haruharu WONDER Black Rice Hyal...,USD 37.00,USD 15.99
...,...,...,...
95,BENTON Aloe BHA Skin Toner 200ml,USD 19.00,USD 16.15
96,COSRX BHA BLACKHEAD POWER LIQUID 100 ML,USD 20.81,USD 12.99
97,SKIN&LAB Medicica Balancing Toner 180ml,USD 21.63,USD 18.39
98,TIAM Vita B5 Toner 180ml,USD 18.60,USD 15.81


In [44]:
# 수집 데이터 저장
df_fin.to_csv('./crawl_data/beauty_price.csv')